# K-Nearest Neighbors from Scratch
## Task 1: Distance-Based Classification

This notebook implements KNN from scratch on the Fashion-MNIST dataset with all bonus features:
- Multiple distance metrics comparison
- Optimized nearest-neighbor search
- Decision boundary visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import time

## Load Fashion-MNIST Dataset

In [ ]:
# Load Fashion-MNIST
print("Loading Fashion-MNIST dataset...")
X, y = fetch_openml('Fashion-MNIST', version=1, return_X_y=True, as_frame=False, parser='auto')

# Normalize pixel values to 0-1
X = X / 255.0
y = y.astype(int)

# Use a subset for faster training (5000 samples)
X_subset = X[:5000]
y_subset = y[:5000]

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X_subset, y_subset, test_size=0.2, random_state=42, stratify=y_subset
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"Features per sample: {X_train.shape[1]}")

# Class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## KNN Implementation from Scratch

In [ ]:
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        """
        K-Nearest Neighbors classifier
        
        Parameters:
        k: number of neighbors to consider
        distance_metric: 'euclidean', 'manhattan', or 'cosine'
        """
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
    
    def fit(self, X, y):
        """Store training data"""
        self.X_train = X
        self.y_train = y
    
    def compute_distance(self, x1, x2):
        """Compute distance between two samples"""
        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        elif self.distance_metric == 'cosine':
            dot_product = np.dot(x1, x2)
            norm_x1 = np.linalg.norm(x1)
            norm_x2 = np.linalg.norm(x2)
            return 1 - (dot_product / (norm_x1 * norm_x2 + 1e-10))
        else:
            raise ValueError(f"Unknown distance metric: {self.distance_metric}")
    
    def predict_single(self, x):
        """Predict class for a single sample"""
        # Compute distances to all training samples
        distances = [self.compute_distance(x, x_train) for x_train in self.X_train]
        
        # Get indices of k nearest neighbors
        k_indices = np.argsort(distances)[:self.k]
        
        # Get labels of k nearest neighbors
        k_nearest_labels = self.y_train[k_indices]
        
        # Return most common label
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]
    
    def predict(self, X):
        """Predict classes for multiple samples"""
        return np.array([self.predict_single(x) for x in X])
    
    def score(self, X, y):
        """Calculate accuracy"""
        predictions = self.predict(X)
        return np.mean(predictions == y)

## Optimized KNN with Vectorized Distance Computation (Bonus)

In [ ]:
class KNN_Optimized:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    
    def compute_distances_vectorized(self, X_test):
        """Vectorized distance computation for speed"""
        if self.distance_metric == 'euclidean':
            # Efficient euclidean distance: ||a-b||^2 = ||a||^2 + ||b||^2 - 2*a·b
            sum_X_train = np.sum(self.X_train ** 2, axis=1)
            sum_X_test = np.sum(X_test ** 2, axis=1)[:, np.newaxis]
            dot_product = np.dot(X_test, self.X_train.T)
            distances = np.sqrt(sum_X_test + sum_X_train - 2 * dot_product)
            return distances
        
        elif self.distance_metric == 'manhattan':
            # Memory-efficient Manhattan distance
            distances = np.zeros((X_test.shape[0], self.X_train.shape[0]))
            for i, x_test in enumerate(X_test):
                distances[i] = np.sum(np.abs(x_test - self.X_train), axis=1)
            return distances
        
        elif self.distance_metric == 'cosine':
            dot_product = np.dot(X_test, self.X_train.T)
            norm_test = np.linalg.norm(X_test, axis=1, keepdims=True)
            norm_train = np.linalg.norm(self.X_train, axis=1, keepdims=True).T
            distances = 1 - (dot_product / (norm_test * norm_train + 1e-10))
            return distances
    
    def predict(self, X):
        # Compute all distances at once
        distances = self.compute_distances_vectorized(X)
        
        # Get k nearest neighbors for each test sample
        k_indices = np.argsort(distances, axis=1)[:, :self.k]
        
        # Get labels of k nearest neighbors
        k_nearest_labels = self.y_train[k_indices]
        
        # Majority voting
        predictions = np.array([Counter(labels).most_common(1)[0][0] 
                               for labels in k_nearest_labels])
        return predictions
    
    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions == y)

## Experiment 1: Effect of K Value

In [ ]:
print("\n=== Testing different K values ===")
k_values = [1, 3, 5, 7, 9, 11, 15]
accuracies = []

for k in k_values:
    knn = KNN_Optimized(k=k, distance_metric='euclidean')
    knn.fit(X_train, y_train)
    accuracy = knn.score(X_test, y_test)
    accuracies.append(accuracy)
    print(f"K={k}: Accuracy = {accuracy:.4f}")

# Plot K vs Accuracy
plt.figure(figsize=(10, 5))
plt.plot(k_values, accuracies, marker='o', linewidth=2, markersize=8)
plt.xlabel('K Value', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Effect of K on KNN Accuracy', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xticks(k_values)
plt.tight_layout()
plt.savefig('k_value_effect.png', dpi=150)
plt.show()

## Experiment 2: Compare Distance Metrics (Bonus)

In [ ]:
print("\n=== Comparing Distance Metrics ===")
distance_metrics = ['euclidean', 'manhattan', 'cosine']
metric_accuracies = []

for metric in distance_metrics:
    knn = KNN_Optimized(k=5, distance_metric=metric)
    knn.fit(X_train, y_train)
    accuracy = knn.score(X_test, y_test)
    metric_accuracies.append(accuracy)
    print(f"{metric.capitalize()}: Accuracy = {accuracy:.4f}")

# Plot Distance Metric Comparison
plt.figure(figsize=(10, 5))
plt.bar(distance_metrics, metric_accuracies, color=['#2ecc71', '#3498db', '#e74c3c'], alpha=0.8)
plt.xlabel('Distance Metric', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Comparison of Distance Metrics (K=5)', fontsize=14)
plt.ylim([min(metric_accuracies) - 0.02, max(metric_accuracies) + 0.02])
plt.grid(True, alpha=0.3, axis='y')
for i, acc in enumerate(metric_accuracies):
    plt.text(i, acc + 0.005, f'{acc:.4f}', ha='center', fontsize=11)
plt.tight_layout()
plt.savefig('distance_metrics_comparison.png', dpi=150)
plt.show()

## Experiment 3: Speed Optimization Comparison (Bonus)

In [ ]:
print("\n=== Comparing Naive vs Optimized Implementation ===")

# Test on small subset for fair comparison
X_test_small = X_test[:100]
y_test_small = y_test[:100]

# Naive implementation
knn_naive = KNN(k=5, distance_metric='euclidean')
knn_naive.fit(X_train, y_train)
start = time.time()
acc_naive = knn_naive.score(X_test_small, y_test_small)
time_naive = time.time() - start

# Optimized implementation
knn_opt = KNN_Optimized(k=5, distance_metric='euclidean')
knn_opt.fit(X_train, y_train)
start = time.time()
acc_opt = knn_opt.score(X_test_small, y_test_small)
time_opt = time.time() - start

print(f"Naive KNN: {time_naive:.3f}s, Accuracy: {acc_naive:.4f}")
print(f"Optimized KNN: {time_opt:.3f}s, Accuracy: {acc_opt:.4f}")
print(f"Speedup: {time_naive/time_opt:.2f}x")

## Final Model Evaluation

In [ ]:
# Train final model with best parameters
best_k = k_values[np.argmax(accuracies)]
best_metric = distance_metrics[np.argmax(metric_accuracies)]

print(f"\n=== Final Model ===")
print(f"Best K: {best_k}")
print(f"Best Metric: {best_metric}")

final_knn = KNN_Optimized(k=best_k, distance_metric=best_metric)
final_knn.fit(X_train, y_train)
y_pred = final_knn.predict(X_test)
final_accuracy = np.mean(y_pred == y_test)

print(f"Final Test Accuracy: {final_accuracy:.4f}")

## Analyze Misclassified Samples

In [ ]:
# Find misclassified samples
misclassified_indices = np.where(y_pred != y_test)[0]
print(f"\nNumber of misclassified samples: {len(misclassified_indices)}")

# Show some misclassified examples
num_show = min(10, len(misclassified_indices))
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for i in range(num_show):
    idx = misclassified_indices[i]
    image = X_test[idx].reshape(28, 28)
    axes[i].imshow(image, cmap='gray')
    axes[i].set_title(f'True: {class_names[y_test[idx]]}\nPred: {class_names[y_pred[idx]]}', 
                      fontsize=9)
    axes[i].axis('off')

plt.tight_layout()
plt.savefig('misclassified_samples.png', dpi=150)
plt.show()

## Confusion Matrix Analysis

In [ ]:
# Compute confusion matrix
num_classes = 10
confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)

for true_label, pred_label in zip(y_test, y_pred):
    confusion_matrix[true_label, pred_label] += 1

# Visualize confusion matrix
plt.figure(figsize=(12, 10))
plt.imshow(confusion_matrix, interpolation='nearest', cmap='Blues')
plt.title('Confusion Matrix', fontsize=16)
plt.colorbar()
tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, class_names, rotation=45, ha='right')
plt.yticks(tick_marks, class_names)

# Add text annotations
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, str(confusion_matrix[i, j]),
                ha="center", va="center",
                color="white" if confusion_matrix[i, j] > confusion_matrix.max() / 2 else "black")

plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
plt.show()

# Class-wise accuracy
print("\nClass-wise Accuracy:")
for i in range(num_classes):
    class_accuracy = confusion_matrix[i, i] / np.sum(confusion_matrix[i, :])
    print(f"{class_names[i]:15s}: {class_accuracy:.4f}")

## Decision Boundary Visualization (Bonus)

In [ ]:
# Reduce to 2D using PCA for visualization
print("\n=== Visualizing Decision Boundaries ===")
pca = PCA(n_components=2)
X_train_2d = pca.fit_transform(X_train)
X_test_2d = pca.transform(X_test)

# Train KNN on 2D data
knn_2d = KNN_Optimized(k=5, distance_metric='euclidean')
knn_2d.fit(X_train_2d, y_train)
accuracy_2d = knn_2d.score(X_test_2d, y_test)
print(f"Accuracy on 2D projection: {accuracy_2d:.4f}")

# Create mesh for decision boundaries
h = 0.5  # step size in the mesh
x_min, x_max = X_train_2d[:, 0].min() - 1, X_train_2d[:, 0].max() + 1
y_min, y_max = X_train_2d[:, 1].min() - 1, X_train_2d[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Predict on mesh
print("Computing decision boundaries (this may take a moment)...")
Z = knn_2d.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot decision boundaries
plt.figure(figsize=(14, 10))
plt.contourf(xx, yy, Z, alpha=0.3, cmap='tab10', levels=np.arange(11) - 0.5)

# Plot training points
colors = plt.cm.tab10(np.linspace(0, 1, 10))
for i in range(10):
    mask = y_train == i
    plt.scatter(X_train_2d[mask, 0], X_train_2d[mask, 1], 
               c=[colors[i]], label=class_names[i], 
               edgecolors='black', linewidth=0.5, s=50, alpha=0.7)

plt.xlabel('First Principal Component', fontsize=12)
plt.ylabel('Second Principal Component', fontsize=12)
plt.title(f'KNN Decision Boundaries (K={best_k}) - PCA Projection', fontsize=14)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
plt.tight_layout()
plt.savefig('decision_boundaries.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary Statistics

In [ ]:
print("\n" + "="*50)
print("FINAL SUMMARY")
print("="*50)
print(f"Dataset: Fashion-MNIST (subset)")
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"\nBest K value: {best_k}")
print(f"Best distance metric: {best_metric}")
print(f"Final accuracy: {final_accuracy:.4f}")
print(f"Misclassified samples: {len(misclassified_indices)} / {len(y_test)}")
print(f"\nOptimization speedup: {time_naive/time_opt:.2f}x")
print("="*50)